In [1]:
import os

In [2]:
%pwd

'd:\\mjourFinalPorject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mjourFinalPorject'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rashmi0710/Classfication-deep-learning-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rashmi0710"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e711db8832861f8ebed2f9e68e91430cea19b918"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.dogvscatclassfier.constants import *
from src.dogvscatclassfier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/data",
            mlflow_uri="https://dagshub.com/rashmi0710/Classfication-deep-learning-project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import mlflow.exceptions

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        try:
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Check if the experiment exists or create it
            experiment_name = "VGG16ModelExperiment"  # Replace with your experiment name
            if not mlflow.get_experiment_by_name(experiment_name):
                mlflow.create_experiment(experiment_name)
            
            mlflow.set_experiment(experiment_name)
            
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy": self.score[1]}
                )
                # Model registry does not work with file store
                if tracking_url_type_store != "file":
                    # Register the model
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")

        except mlflow.exceptions.MlflowException as e:
            print(f"MLflowException: {e}")
        except Exception as e:
            print(f"Exception: {e}")

In [13]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-05 14:00:33,035: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 14:00:33,048: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 14:00:33,052: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 20s 715ms/step - loss: 0.6830 - accuracy: 0.9137
[2024-07-05 14:00:53,594: INFO: common: json file saved at: scores.json]


2024/07/05 14:00:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-05 14:00:58,826: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\dongr\AppData\Local\Temp\tmpkkmtz2fp\model\data\model\assets
[2024-07-05 14:01:00,916: INFO: builder_impl: Assets written to: C:\Users\dongr\AppData\Local\Temp\tmpkkmtz2fp\model\data\model\assets]


d:\mjourFinalPorject\mf\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


KeyboardInterrupt: 

In [14]:
from mlflow import MlflowClient
client = MlflowClient()
client.transition_model_version_stage(
    name="VGG16Model", version=9, stage="Production"
)

<ModelVersion: creation_timestamp=1720079663645, current_stage='Production', description='this experiment outcome are best.', last_updated_timestamp=1720168417859, name='VGG16Model', run_id='fd9830cf53364486a0aa211c4b78e385', run_link='', source='mlflow-artifacts:/9eb499e1237a454c845da852eae8fdde/fd9830cf53364486a0aa211c4b78e385/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='9'>